# Preparação do banco de dados CAPES

Banco de Dados com informações sobre bolsas de mestrado, doutorado no Brasil de 1995 até 2023

A fonte de dados é GEOCAPES https://geocapes.capes.gov.br/geocapes/ 

## Conhecendo os dados 

In [45]:
# Importação das bibliotecas que são úteis ao longo do notebook 
## Lembrnado, caso não tenha instalado é necessário passar o pip install para os programas 
import pandas as pd
import numpy as np
from unidecode import unidecode

# Configurar para não exibir warnings
import warnings

warnings.filterwarnings("ignore")


In [46]:
capes = pd.read_excel(r'C:\Users\joaov\OneDrive\Documentos\CAPES\dados.xlsx')

In [47]:
capes.head()

,Ano,UF,Município,Região,Código Programa,Programa Fomento,IES,Status Jurídico,Programa,Área Avaliação,Área Conhecimento,Grande Área,Codigo IES,DOUTORADO PLENO,MESTRADO,PROFESSOR VISITANTE,PÓS-DOUTORADO,Total Linha,PESQUISADOR VISITANTE ESPECIAL,PROF VISITANTE NACIONAL SÊNIOR,Iniciação Científica,PROFESSOR VISITANTE EXTERIOR SENIOR,DOUTORADO PROFFISIONAL,JOVENS TALENTOS - A,JOVENS TALENTOS - B,MESTRADO PROFISSIONAL,PROFESSOR VISITANTE EXTERIOR PLENO,Coordenador Geral IsF,Coordenador Pedagógico IsF,Professor IsF,Supervisão,Coordenador de Centro IsF
0,2023,AC,RIO BRANCO,NORTE,NaN,DS,UFAC,Federal,NaN,INDEFINIDA ...,Grande Area Não Informada ...,Grande Area Não Informada ...,11001011.0,6,8,0.0,0.0,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023,AC,RIO BRANCO,NORTE,11001011001P8,CONSOLIDACAO,UFAC,Federal,ECOLOGIA E MANEJO DE RECURSOS NATURAIS,BIODIVERSIDADE ...,ECOLOGIA ...,CIÊNCIAS BIOLÓGICAS ...,11001011.0,0,2,0.0,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023,AC,RIO BRANCO,NORTE,11001011003P0,CONSOLIDACAO,UFAC,Federal,LETRAS- LINGUAGEM E IDENTIDADE,LINGUÍSTICA E LITERATURA,LETRAS ...,"LINGÜÍSTICA, LETRAS E ARTES ...",11001011.0,2,0,0.0,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023,AC,RIO BRANCO,NORTE,11001011003P0,DS,UFAC,Federal,LETRAS- LINGUAGEM E IDENTIDADE,LINGUÍSTICA E LITERATURA,LETRAS ...,"LINGÜÍSTICA, LETRAS E ARTES ...",11001011.0,11,17,0.0,0.0,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023,AC,RIO BRANCO,NORTE,11001011003P0,PDPG-FAPIII,UFAC,Federal,LETRAS- LINGUAGEM E IDENTIDADE,LINGUÍSTICA E LITERATURA,LETRAS ...,"LINGÜÍSTICA, LETRAS E ARTES ...",11001011.0,10,16,0.0,1.0,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Verificando as informações da base de dados 

In [48]:
# Quais informações apresentam informações nulas
capes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180518 entries, 0 to 180517
Data columns (total 32 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   Ano                                  180518 non-null  int64  
 1   UF                                   180518 non-null  object 
 2   Município                            180518 non-null  object 
 3   Região                               180518 non-null  object 
 4   Código Programa                      173378 non-null  object 
 5   Programa Fomento                     180518 non-null  object 
 6   IES                                  180517 non-null  object 
 7   Status Jurídico                      180518 non-null  object 
 8   Programa                             173378 non-null  object 
 9   Área Avaliação                       180518 non-null  object 
 10  Área Conhecimento                    180518 non-null  object 
 11  Grande Área  

In [49]:
capes.columns

Index(['Ano', 'UF', 'Município', 'Região', 'Código Programa',
       'Programa Fomento', 'IES', 'Status Jurídico', 'Programa',
       'Área Avaliação', 'Área Conhecimento', 'Grande Área', 'Codigo IES',
       'DOUTORADO PLENO', 'MESTRADO', 'PROFESSOR VISITANTE', 'PÓS-DOUTORADO',
       'Total Linha', 'PESQUISADOR VISITANTE ESPECIAL',
       'PROF VISITANTE NACIONAL SÊNIOR', 'Iniciação Científica',
       'PROFESSOR VISITANTE EXTERIOR SENIOR', 'DOUTORADO PROFFISIONAL',
       'JOVENS TALENTOS - A', 'JOVENS TALENTOS - B', 'MESTRADO PROFISSIONAL',
       'PROFESSOR VISITANTE EXTERIOR PLENO', 'Coordenador Geral IsF',
       'Coordenador Pedagógico IsF', 'Professor IsF', 'Supervisão',
       'Coordenador de Centro IsF'],
      dtype='object')

In [50]:
## Analíse da relação dos valores faltantes com os anos, para ver se existe relação 
qtd_nulos = pd.DataFrame()
for column in capes.columns[13:]:   ## 13 pois a var de corte para as númericas
    null_counts = capes.groupby('Ano')[column].apply(lambda x: x.isnull().sum())
    non_null_counts = capes.groupby('Ano')[column].apply(lambda x: x.notnull().sum())
    qtd_nulos[f'nulos_{column}'] = null_counts
    qtd_nulos[f'não_nulos_{column}'] = non_null_counts

pd.set_option('display.max_columns', None)
qtd_nulos

,nulos_DOUTORADO PLENO,não_nulos_DOUTORADO PLENO,nulos_MESTRADO,não_nulos_MESTRADO,nulos_PROFESSOR VISITANTE,não_nulos_PROFESSOR VISITANTE,nulos_PÓS-DOUTORADO,não_nulos_PÓS-DOUTORADO,nulos_Total Linha,não_nulos_Total Linha,nulos_PESQUISADOR VISITANTE ESPECIAL,não_nulos_PESQUISADOR VISITANTE ESPECIAL,nulos_PROF VISITANTE NACIONAL SÊNIOR,não_nulos_PROF VISITANTE NACIONAL SÊNIOR,nulos_Iniciação Científica,não_nulos_Iniciação Científica,nulos_PROFESSOR VISITANTE EXTERIOR SENIOR,não_nulos_PROFESSOR VISITANTE EXTERIOR SENIOR,nulos_DOUTORADO PROFFISIONAL,não_nulos_DOUTORADO PROFFISIONAL,nulos_JOVENS TALENTOS - A,não_nulos_JOVENS TALENTOS - A,nulos_JOVENS TALENTOS - B,não_nulos_JOVENS TALENTOS - B,nulos_MESTRADO PROFISSIONAL,não_nulos_MESTRADO PROFISSIONAL,nulos_PROFESSOR VISITANTE EXTERIOR PLENO,não_nulos_PROFESSOR VISITANTE EXTERIOR PLENO,nulos_Coordenador Geral IsF,não_nulos_Coordenador Geral IsF,nulos_Coordenador Pedagógico IsF,não_nulos_Coordenador Pedagógico IsF,nulos_Professor IsF,não_nulos_Professor IsF,nulos_Supervisão,não_nulos_Supervisão,nulos_Coordenador de Centro IsF,não_nulos_Coordenador de Centro IsF
Ano,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1995,0,1723,0,1723,1723,0,1723,0,0,1723,1723,0,1723,0,1723,0,1723,0,1723,0,1723,0,1723,0,1723,0,1723,0,1723,0,1723,0,1723,0,1723,0,1723,0
1996,0,1810,0,1810,1810,0,1810,0,0,1810,1810,0,1810,0,1810,0,1810,0,1810,0,1810,0,1810,0,1810,0,1810,0,1810,0,1810,0,1810,0,1810,0,1810,0
1997,0,1926,0,1926,1926,0,1926,0,0,1926,1926,0,1926,0,1926,0,1926,0,1926,0,1926,0,1926,0,1926,0,1926,0,1926,0,1926,0,1926,0,1926,0,1926,0
1998,0,1977,0,1977,1977,0,1977,0,0,1977,1977,0,1977,0,1977,0,1977,0,1977,0,1977,0,1977,0,1977,0,1977,0,1977,0,1977,0,1977,0,1977,0,1977,0
1999,0,1965,0,1965,1965,0,1965,0,0,1965,1965,0,1965,0,1965,0,1965,0,1965,0,1965,0,1965,0,1965,0,1965,0,1965,0,1965,0,1965,0,1965,0,1965,0
2000,0,2175,0,2175,2175,0,0,2175,0,2175,2175,0,2175,0,2175,0,2175,0,2175,0,2175,0,2175,0,2175,0,2175,0,2175,0,2175,0,2175,0,2175,0,2175,0
2001,0,2338,0,2338,2338,0,0,2338,0,2338,2338,0,2338,0,2338,0,2338,0,2338,0,2338,0,2338,0,2338,0,2338,0,2338,0,2338,0,2338,0,2338,0,2338,0
2002,0,2606,0,2606,2606,0,0,2606,0,2606,2606,0,2606,0,2606,0,2606,0,2606,0,2606,0,2606,0,2606,0,2606,0,2606,0,2606,0,2606,0,2606,0,2606,0
2003,0,4390,0,4390,4390,0,0,4390,0,4390,4390,0,4390,0,4390,0,4390,0,4390,0,4390,0,4390,0,4390,0,4390,0,4390,0,4390,0,4390,0,4390,0,4390,0


Fazendo a análise da quantidade de nulos por ano para cada variável percebemos que temos algumas que passaram a ser contados somente em alguns anos, Como o código do IES (Instituição de Ensino Superior), professores visitantes do exterior. E algumas variáveis são contadas somente apartir de um momento, como o Coordenador Geral do ISF e informações de Iniciação Cientifica tem somente dados de 2013 para frente, Coordenador Pedagógico IsF de 2014, entre outras informações que só aparecem depois de um certo momento, logo se essas informações forem objeto de estudo é interessante separar para somente aqueles anos em questão.

In [51]:
## Transformação dos faltantes em relação aos valores inteiros, para zero.
colunas_0 = ['Codigo IES',
       'Coordenador Geral IsF', 'Coordenador Pedagógico IsF','Iniciação Científica', 'JOVENS TALENTOS - A',
       'JOVENS TALENTOS - B', 'MESTRADO PROFISSIONAL',
       'PESQUISADOR VISITANTE ESPECIAL', 'PROF VISITANTE NACIONAL SÊNIOR',
       'PROFESSOR VISITANTE EXTERIOR PLENO',
       'PROFESSOR VISITANTE EXTERIOR SENIOR', 'Professor IsF', 'PÓS-DOUTORADO', 'PROFESSOR VISITANTE',
       'DOUTORADO PROFFISIONAL', 'Supervisão', 'Coordenador de Centro IsF' ]
## Transformando os NA em Zero, mostrando como pode ser feito
capes[colunas_0] = capes[colunas_0].fillna(0)
capes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180518 entries, 0 to 180517
Data columns (total 32 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   Ano                                  180518 non-null  int64  
 1   UF                                   180518 non-null  object 
 2   Município                            180518 non-null  object 
 3   Região                               180518 non-null  object 
 4   Código Programa                      173378 non-null  object 
 5   Programa Fomento                     180518 non-null  object 
 6   IES                                  180517 non-null  object 
 7   Status Jurídico                      180518 non-null  object 
 8   Programa                             173378 non-null  object 
 9   Área Avaliação                       180518 non-null  object 
 10  Área Conhecimento                    180518 non-null  object 
 11  Grande Área  

Tem um faltante em IES, para corrigir o IES podemos encontrar ele no dataframe e comparar ver de qual múnicipio ele é, e quais IES tem aquele municipio.

In [52]:
capes[capes['IES'].isnull()]
## Somente em Santarém no Pará, logo podemos procurar para ver todas as IES de Santarém

,Ano,UF,Município,Região,Código Programa,Programa Fomento,IES,Status Jurídico,Programa,Área Avaliação,Área Conhecimento,Grande Área,Codigo IES,DOUTORADO PLENO,MESTRADO,PROFESSOR VISITANTE,PÓS-DOUTORADO,Total Linha,PESQUISADOR VISITANTE ESPECIAL,PROF VISITANTE NACIONAL SÊNIOR,Iniciação Científica,PROFESSOR VISITANTE EXTERIOR SENIOR,DOUTORADO PROFFISIONAL,JOVENS TALENTOS - A,JOVENS TALENTOS - B,MESTRADO PROFISSIONAL,PROFESSOR VISITANTE EXTERIOR PLENO,Coordenador Geral IsF,Coordenador Pedagógico IsF,Professor IsF,Supervisão,Coordenador de Centro IsF
47850,2018,PA,SANTARÉM,NORTE,33003017001P2,PROCAD,NaN,Federal,EDUCAÇÃO,INDEFINIDA ...,EDUCAÇÃO ...,CIÊNCIAS HUMANAS ...,15010023.0,0,0,0.0,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
capes.loc[capes['Município'] == 'SANTARÉM', 'IES'].unique()
## Como só temos UFOPA e UFOPA Santarém, sao duas informações vinda da mesma faculdade, UFOPA de Santarem
## Logo irei substituir por ufopa somente e depois substituir todas ufopa santarem por ufopa, já que eu tenho o municipio, mais para frente posso utilizar
## essa var para distinguir entre as ufopas 
capes['IES'].replace({np.nan: 'UFOPA', 'UFOPA Santarém': 'UFOPA'}, inplace=True)

## Correção de caracteres 

In [54]:
# Para transformar os caracteres especiais em normais e transformando tudo em upper para não ocorrer em erros de alguma linha ta e outra não 
capes = capes.applymap(lambda x: unidecode(str(x)) if isinstance(x, str) else x)
capes = capes.applymap(lambda x: str(x).upper() if isinstance(x, str) else x)

In [55]:
capes.head()

,Ano,UF,Município,Região,Código Programa,Programa Fomento,IES,Status Jurídico,Programa,Área Avaliação,Área Conhecimento,Grande Área,Codigo IES,DOUTORADO PLENO,MESTRADO,PROFESSOR VISITANTE,PÓS-DOUTORADO,Total Linha,PESQUISADOR VISITANTE ESPECIAL,PROF VISITANTE NACIONAL SÊNIOR,Iniciação Científica,PROFESSOR VISITANTE EXTERIOR SENIOR,DOUTORADO PROFFISIONAL,JOVENS TALENTOS - A,JOVENS TALENTOS - B,MESTRADO PROFISSIONAL,PROFESSOR VISITANTE EXTERIOR PLENO,Coordenador Geral IsF,Coordenador Pedagógico IsF,Professor IsF,Supervisão,Coordenador de Centro IsF
0,2023,AC,RIO BRANCO,NORTE,NaN,DS,UFAC,FEDERAL,NaN,INDEFINIDA ...,GRANDE AREA NAO INFORMADA ...,GRANDE AREA NAO INFORMADA ...,11001011.0,6,8,0.0,0.0,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023,AC,RIO BRANCO,NORTE,11001011001P8,CONSOLIDACAO,UFAC,FEDERAL,ECOLOGIA E MANEJO DE RECURSOS NATURAIS,BIODIVERSIDADE ...,ECOLOGIA ...,CIENCIAS BIOLOGICAS ...,11001011.0,0,2,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023,AC,RIO BRANCO,NORTE,11001011003P0,CONSOLIDACAO,UFAC,FEDERAL,LETRAS- LINGUAGEM E IDENTIDADE,LINGUISTICA E LITERATURA,LETRAS ...,"LINGUISTICA, LETRAS E ARTES ...",11001011.0,2,0,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023,AC,RIO BRANCO,NORTE,11001011003P0,DS,UFAC,FEDERAL,LETRAS- LINGUAGEM E IDENTIDADE,LINGUISTICA E LITERATURA,LETRAS ...,"LINGUISTICA, LETRAS E ARTES ...",11001011.0,11,17,0.0,0.0,28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023,AC,RIO BRANCO,NORTE,11001011003P0,PDPG-FAPIII,UFAC,FEDERAL,LETRAS- LINGUAGEM E IDENTIDADE,LINGUISTICA E LITERATURA,LETRAS ...,"LINGUISTICA, LETRAS E ARTES ...",11001011.0,10,16,0.0,1.0,27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
## Verificando se algum caracter diferente seguiu na tabela
def encontrar_caracteres_especiais(df):
    caracteres_especiais = []
    for coluna in df.columns:
        for valor in df[coluna]:
            if isinstance(valor, str) and not valor.isascii():
                caracteres_especiais.append((coluna, valor))
    return caracteres_especiais

caracteres_problema = encontrar_caracteres_especiais(capes)
print(caracteres_problema)
## Deu vazio o unicode limpou todas os caracteres especiais

[]


## Corrigindo o nome das colunas para nomes sem acentos e espaço

In [57]:
colunas_capes = {
    'Ano': 'ANO',
    'UF': 'UF',  
    'Município': 'MUN',
    'Região': 'REGIAO',
    'Código Programa': 'COD_PROG',
    'Programa Fomento': 'PROG_FORMENTO',
    'IES': 'IES',  
    'Status Jurídico': 'STATUS_JURIDICO',
    'Programa': 'PROGRAMA',
    'Área Avaliação': 'AREA_AVALIACAO',
    'Área Conhecimento': 'AREA_CONHECIMENTO',
    'Grande Área': 'GRANDE_AREA',
    'Codigo IES': 'COD_IES',  
    'DOUTORADO PLENO': 'DOUTORADO',
    'MESTRADO': 'MESTRADO', 
    'PROFESSOR VISITANTE': 'PROF_VISITANTE', 
    'PÓS-DOUTORADO': 'POS_DOUTORADO',  
    'Total Linha': 'TOTAL_LINHA', 
    'PESQUISADOR VISITANTE ESPECIAL': 'PESQ_VISITANTE_ESP',  
    'PROF VISITANTE NACIONAL SÊNIOR': 'PROF_VISITANTE_NAC_SENIOR',  
    'Iniciação Científica': 'INICIACAO_CIENTIFICA',  
    'PROFESSOR VISITANTE EXTERIOR SENIOR': 'PROF_VISITANTE_EXT_SENIOR', 
    'DOUTORADO PROFFISIONAL': 'DOUTORADO_PROFISSIONAL',  
    'JOVENS TALENTOS - A': 'JOVENS_TALENTOS_A', 
    'JOVENS TALENTOS - B': 'JOVENS_TALENTOS_B',  
    'MESTRADO PROFISSIONAL': 'MESTRADO_PROFISSIONAL', 
    'PROFESSOR VISITANTE EXTERIOR PLENO': 'PROF_VISITANTE_EXT_PLENO',  
    'Coordenador Geral IsF': 'COORDENADOR_GERAL_ISF', 
    'Coordenador Pedagógico IsF': 'COORDENADOR_PEDAGOGICO_ISF',  
    'Professor IsF': 'PROFESSOR_ISF',  
    'Supervisão': 'SUPERVISAO',  
    'Coordenador de Centro IsF': 'COORDENADOR_CENTRO_ISF'  
}
capes.rename(columns=colunas_capes, inplace=True)

In [58]:
capes.columns

Index(['ANO', 'UF', 'MUN', 'REGIAO', 'COD_PROG', 'PROG_FORMENTO', 'IES',
       'STATUS_JURIDICO', 'PROGRAMA', 'AREA_AVALIACAO', 'AREA_CONHECIMENTO',
       'GRANDE_AREA', 'COD_IES', 'DOUTORADO', 'MESTRADO', 'PROF_VISITANTE',
       'POS_DOUTORADO', 'TOTAL_LINHA', 'PESQ_VISITANTE_ESP',
       'PROF_VISITANTE_NAC_SENIOR', 'INICIACAO_CIENTIFICA',
       'PROF_VISITANTE_EXT_SENIOR', 'DOUTORADO_PROFISSIONAL',
       'JOVENS_TALENTOS_A', 'JOVENS_TALENTOS_B', 'MESTRADO_PROFISSIONAL',
       'PROF_VISITANTE_EXT_PLENO', 'COORDENADOR_GERAL_ISF',
       'COORDENADOR_PEDAGOGICO_ISF', 'PROFESSOR_ISF', 'SUPERVISAO',
       'COORDENADOR_CENTRO_ISF'],
      dtype='object')

## Corrigindo as IES 
No banco de dados não existe um padrão para os nomes dos IES, pois pela caractéristica das universidades brasileiras apresentarem campus em diferentes cidades podemos ter diferentes nomes para a mesma instituição, um exemplo é como mostrado abaixo, o Município de Areia na Paraíba tem um campus da UFPB, porém quando olhamos os IES do município temos a UFPB e UFPB/ARREIA, que são a mesma universidade, dessa forma irei padronizar o nome das universidades para apenas UFPB, e quando for o objetivo olhar o campus em questão pode-se agrupar com o município.

In [59]:
capes.loc[capes['MUN'] == 'AREIA', 'IES'].unique()

array(['UFPB/AREIA', 'UFPB'], dtype=object)

In [60]:
capes['MUN'].unique()

array(['RIO BRANCO', 'ARAPIRACA', 'MACEIO', 'MANAUS', 'MACAPA',
       'BARREIRAS', 'CRUZ DAS ALMAS', 'FEIRA DE SANTANA', 'GUANAMBI',
       'ILHEUS', 'ITABUNA', 'SALVADOR', 'SAO FRANCISCO DO CONDE',
       'VITORIA DA CONQUISTA', 'CRATO', 'FORTALEZA', 'JUAZEIRO DO NORTE',
       'REDENCAO', 'SOBRAL', 'BRASILIA', 'VILA VELHA', 'VITORIA',
       'ANAPOLIS', 'CATALAO', 'GOIANIA', 'INHUMAS', 'JATAI', 'RIO VERDE',
       'IMPERATRIZ', 'SAO LUIS', 'ALFENAS', 'BELO HORIZONTE',
       'DIAMANTINA', 'GOVERNADOR VALADARES', 'ITAJUBA', 'ITAUNA',
       'JUIZ DE FORA', 'LAVRAS', 'MONTES CLAROS', 'NOVA LIMA',
       'OURO PRETO', 'POUSO ALEGRE', 'SANTA RITA DO SAPUCAI',
       'SAO JOAO DEL REI', 'UBERABA', 'UBERLANDIA', 'VARGINHA', 'VICOSA',
       'CAMPO GRANDE', 'DOURADOS', 'CUIABA', 'CACERES', 'RONDONOPOLIS',
       'VARZEA GRANDE', 'ANANINDEUA', 'BELEM', 'MARABA', 'SANTAREM',
       'AREIA', 'CAMPINA GRANDE', 'JOAO PESSOA', 'RIO TINTO', 'PETROLINA',
       'RECIFE', 'PARNAIBA', 'TERESINA', 'C

In [61]:
capes['IES'].unique()

array(['UFAC', 'UNEAL', 'UFAL', 'CESMAC', 'UNIT/ALAGOAS',
       'FIOCRUZ-CPQLMD', 'INPA', 'UEA', 'UFAM', 'UNINILTON', 'UNIFAP',
       'UFOB', 'UFRB', 'UEFS', 'UNIFG', 'UESC', 'UFSB', 'UFBA', 'UNEB',
       'UCSAL', 'UNIFACS', 'EBMSP', 'SENAI-CIMATEC', 'FIOCRUZ-CPQGM',
       'IFBAIANO', 'UNILAB  -MALES', 'UESB', 'URCA', 'IFCE', 'UECE',
       'UFC', 'UNICHRISTUS', 'UNIFOR', 'UNI7', 'FRT', 'UFCA', 'UNILAB',
       'UVA-CE', 'UNINTA', 'UNB', 'UCB-TAG', 'UNICEUB', 'IDP',
       'FEPECS-ESCS', 'UDF', 'FGV/BSB', 'UVV', 'UFES', 'IFES', 'FUCAPE',
       'FDV', 'UEG', 'UNIEVANGELICA', 'UFCAT', 'IFGOIANO', 'UFG',
       'PUC-GOIAS', 'IFG', 'UNIALFA', 'FACMAIS', 'UFJ', 'UNIRV',
       'UEMASUL', 'IFMA', 'UEMA', 'UFMA', 'UNICEUMA', 'IFMA - MC',
       'FUCAPE-MA', 'UNIFAL-MG', 'UNIFENAS', 'CEFET-MG', 'UEMG', 'UFMG',
       'PUC/MG', 'FUMEC', 'FAJE', 'FCMMG', 'FJP', 'FSCBH',
       'FIOCRUZ-CPQRR', 'CDTN', 'UNIHORIZONTES', 'ESDHC', 'UFVJM',
       'UFJF-GV', 'UNIVALE', 'UNIFEI', 'UI', 'UFJF', 'U

In [62]:
# vimos que temos universidades que estão com - e outras com /, transformar tudo como /
capes['IES'] = capes['IES'].str.replace('-', '/').str.replace('/', '/').str.replace(' ','/')

In [63]:
## para mantermos somente o IES puro, sem o nome do campos, o campos iremos utilizar pelo municipio, pois existem momentos
## no banco de dados que alguns tem campos e outros não, o que está diferenciando a mesmo IES para isso iremos padronizar e olhar os campos pelo MUN
capes['IES'] = capes['IES'].str.split('/').str[0]

In [64]:
capes.head()

,ANO,UF,MUN,REGIAO,COD_PROG,PROG_FORMENTO,IES,STATUS_JURIDICO,PROGRAMA,AREA_AVALIACAO,AREA_CONHECIMENTO,GRANDE_AREA,COD_IES,DOUTORADO,MESTRADO,PROF_VISITANTE,POS_DOUTORADO,TOTAL_LINHA,PESQ_VISITANTE_ESP,PROF_VISITANTE_NAC_SENIOR,INICIACAO_CIENTIFICA,PROF_VISITANTE_EXT_SENIOR,DOUTORADO_PROFISSIONAL,JOVENS_TALENTOS_A,JOVENS_TALENTOS_B,MESTRADO_PROFISSIONAL,PROF_VISITANTE_EXT_PLENO,COORDENADOR_GERAL_ISF,COORDENADOR_PEDAGOGICO_ISF,PROFESSOR_ISF,SUPERVISAO,COORDENADOR_CENTRO_ISF
0,2023,AC,RIO BRANCO,NORTE,NaN,DS,UFAC,FEDERAL,NaN,INDEFINIDA ...,GRANDE AREA NAO INFORMADA ...,GRANDE AREA NAO INFORMADA ...,11001011.0,6,8,0.0,0.0,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023,AC,RIO BRANCO,NORTE,11001011001P8,CONSOLIDACAO,UFAC,FEDERAL,ECOLOGIA E MANEJO DE RECURSOS NATURAIS,BIODIVERSIDADE ...,ECOLOGIA ...,CIENCIAS BIOLOGICAS ...,11001011.0,0,2,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023,AC,RIO BRANCO,NORTE,11001011003P0,CONSOLIDACAO,UFAC,FEDERAL,LETRAS- LINGUAGEM E IDENTIDADE,LINGUISTICA E LITERATURA,LETRAS ...,"LINGUISTICA, LETRAS E ARTES ...",11001011.0,2,0,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023,AC,RIO BRANCO,NORTE,11001011003P0,DS,UFAC,FEDERAL,LETRAS- LINGUAGEM E IDENTIDADE,LINGUISTICA E LITERATURA,LETRAS ...,"LINGUISTICA, LETRAS E ARTES ...",11001011.0,11,17,0.0,0.0,28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023,AC,RIO BRANCO,NORTE,11001011003P0,PDPG-FAPIII,UFAC,FEDERAL,LETRAS- LINGUAGEM E IDENTIDADE,LINGUISTICA E LITERATURA,LETRAS ...,"LINGUISTICA, LETRAS E ARTES ...",11001011.0,10,16,0.0,1.0,27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Vendo se existe problema na nomeação dos Estados, Regiões e do Status_Juridico

In [65]:
capes['UF'].unique()

array(['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG',
       'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR',
       'RS', 'SC', 'SE', 'SP', 'TO'], dtype=object)

In [66]:
capes['REGIAO'].unique()

array(['NORTE', 'NORDESTE', 'CENTRO-OESTE', 'SUDESTE', 'SUL'],
      dtype=object)

In [67]:
capes['STATUS_JURIDICO'].unique()

array(['FEDERAL', 'ESTADUAL', 'PRIVADA', 'MUNICIPAL'], dtype=object)

Todas as três não apresentam informações com nome diferentes

In [68]:
# Exportação do banco
capes.to_csv('capes.csv', index=False)